# Packages

To install rapidsai packages on Google Colab, you need to execute the following code:

In [1]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 460, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 460 (delta 131), reused 124 (delta 91), pack-reused 269
Receiving objects: 100% (460/460), 126.19 KiB | 18.03 MiB/s, done.
Resolving deltas: 100% (233/233), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.5 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS via pip!  Please stand by, should be quick...
***********************************************************************

***********************************************************************
Woo! Your instance has a Tesla T4 GPU!
We will install the latest stable RAPIDS via pip 24.2.*!  Please stand by, should be quick...
***********************************************************************

Looking in in

Upgrade the setuptools and pip packages to their latest versions

In [2]:
!python -m pip install -U setuptools pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.6 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [3]:
!pip uninstall cupy-cuda11x -y
!pip install cupy-cuda12x

In [4]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [5]:
import cudf
import cupy as cp

In [6]:
!pip install gdown==4.6.0

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [7]:
!gdown 1-EXISwPY0afVl-6a4Jzmar9vcVMuTG7A #trainingData.csv
!gdown 1-QzHR3ou-6ipEqf_cM1NgkQGgqShftOS #validationData.csv

Downloading...
From: https://drive.google.com/uc?id=1-EXISwPY0afVl-6a4Jzmar9vcVMuTG7A
To: /content/trainingData.csv
100% 42.7M/42.7M [00:00<00:00, 47.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-QzHR3ou-6ipEqf_cM1NgkQGgqShftOS
To: /content/validationData.csv
100% 2.38M/2.38M [00:00<00:00, 66.6MB/s]


# Offline

In [49]:
BUILDINGID=2 # 0 , 1 , 2
FLOOR=0      # 0 , 1 , 2 , 3

In [50]:
df_train = cudf.read_csv("/content/trainingData.csv")
df_val = cudf.read_csv("/content/validationData.csv")

In [51]:
grouped = df_train.groupby(['BUILDINGID'])
num_unique_groups = len(grouped.agg('size'))

def get_records_by_number(group_number):
    group_key = list(grouped.groups.keys())[group_number]
    return grouped.get_group(group_key)

output_directory = 'building_csv_files'
os.makedirs(output_directory, exist_ok=True)

for group_number in range(num_unique_groups):
    records_for_group = get_records_by_number(group_number)
    if not records_for_group.empty:
        output_file_path = os.path.join(output_directory, f'train_building_{group_number}.csv')

        records_np = cp.asnumpy(records_for_group.values)

        pd.DataFrame(records_np, columns=records_for_group.columns).to_csv(output_file_path, index=False)

        print(f'Group {group_number} records saved to {output_file_path}')


Group 0 records saved to building_csv_files/train_building_0.csv
Group 1 records saved to building_csv_files/train_building_1.csv
Group 2 records saved to building_csv_files/train_building_2.csv


In [52]:
df_train_temp = cudf.read_csv(f"/content/building_csv_files/train_building_{BUILDINGID}.csv")
df_train_temp['FLOOR'] = df_train_temp['FLOOR'].astype(int)
grouped_by_floor = df_train_temp.groupby('FLOOR')

output_directory = '/content/floor_csv_files'
os.makedirs(output_directory, exist_ok=True)

for floor, group in grouped_by_floor:
    output_file_path = os.path.join(output_directory, f'train_B{BUILDINGID}_F{floor}.csv')
    group.to_csv(output_file_path, index=False)
    print(f'Floor {floor} records saved to {output_file_path}')


Floor 0 records saved to /content/floor_csv_files/train_B2_F0.csv
Floor 1 records saved to /content/floor_csv_files/train_B2_F1.csv
Floor 2 records saved to /content/floor_csv_files/train_B2_F2.csv
Floor 3 records saved to /content/floor_csv_files/train_B2_F3.csv
Floor 4 records saved to /content/floor_csv_files/train_B2_F4.csv


In [53]:
grouped = df_val.groupby(['BUILDINGID'])
num_unique_groups = len(grouped.agg('size'))

def get_records_by_number(group_number):
    group_key = list(grouped.groups.keys())[group_number]
    return grouped.get_group(group_key)

output_directory = 'building_csv_files'
os.makedirs(output_directory, exist_ok=True)

for group_number in range(num_unique_groups):
    records_for_group = get_records_by_number(group_number)
    if not records_for_group.empty:
        output_file_path = os.path.join(output_directory, f'val_building_{group_number}.csv')

        records_np = cp.asnumpy(records_for_group.values)

        pd.DataFrame(records_np, columns=records_for_group.columns).to_csv(output_file_path, index=False)

        print(f'Group {group_number} records saved to {output_file_path}')


Group 0 records saved to building_csv_files/val_building_0.csv
Group 1 records saved to building_csv_files/val_building_1.csv
Group 2 records saved to building_csv_files/val_building_2.csv


In [56]:
df_val_temp = cudf.read_csv(f"/content/building_csv_files/val_building_{BUILDINGID}.csv")
df_val_temp['FLOOR'] = df_val_temp['FLOOR'].astype(int)
grouped_by_floor = df_val_temp.groupby('FLOOR')

output_directory = '/content/floor_csv_files'
os.makedirs(output_directory, exist_ok=True)

for floor, group in grouped_by_floor:
    output_file_path = os.path.join(output_directory, f'val_B{BUILDINGID}_F{floor}.csv')
    group.to_csv(output_file_path, index=False)
    print(f'Floor {floor} records saved to {output_file_path}')


Floor 0 records saved to /content/floor_csv_files/val_B2_F0.csv
Floor 1 records saved to /content/floor_csv_files/val_B2_F1.csv
Floor 2 records saved to /content/floor_csv_files/val_B2_F2.csv
Floor 3 records saved to /content/floor_csv_files/val_B2_F3.csv
Floor 4 records saved to /content/floor_csv_files/val_B2_F4.csv


In [57]:
df_train_test = cudf.read_csv(f"/content/floor_csv_files/train_B{BUILDINGID}_F{FLOOR}.csv")
df_train_test_temp = df_train_test.iloc[:, :520].replace(100, -104)
df_train_test = cudf.concat([df_train_test_temp, df_train_test.iloc[:, 520:]], axis=1)
df_train_test.to_csv("New_trainingData.csv", index=False)
df_train_test.shape

(1942, 529)

In [58]:
df_val_test = cudf.read_csv(f"/content/floor_csv_files/val_B{BUILDINGID}_F{FLOOR}.csv")
df_val_test_temp = df_val_test.iloc[:, :520].replace(100, -104)
df_val_test = cudf.concat([df_val_test_temp, df_val_test.iloc[:, 520:]], axis=1)
df_val_test.to_csv("New_validationData.csv", index=False)
df_val_test.shape

(24, 529)

In [59]:
df_train = cudf.read_csv("New_trainingData.csv")
grouped_cols = df_train[['LONGITUDE', 'LATITUDE', 'FLOOR']]
grouped = df_train.groupby([col for col in grouped_cols.columns])

num_unique_groups = len(grouped.agg('size'))

def get_records_by_number(group_number):
    group_key = list(grouped.groups.keys())[group_number]
    return grouped.get_group(group_key)

output_directory = 'RP'
os.makedirs(output_directory, exist_ok=True)

for group_number in range(num_unique_groups):
    records_for_group = get_records_by_number(group_number)

    if not records_for_group.empty:
        output_file_path = os.path.join(output_directory, f'RP_{group_number}.csv')
        records_for_group.to_csv(output_file_path, index=False)
        print(f'Group {group_number} records saved to {output_file_path}')


Group 0 records saved to RP/RP_0.csv
Group 1 records saved to RP/RP_1.csv
Group 2 records saved to RP/RP_2.csv
Group 3 records saved to RP/RP_3.csv
Group 4 records saved to RP/RP_4.csv
Group 5 records saved to RP/RP_5.csv
Group 6 records saved to RP/RP_6.csv
Group 7 records saved to RP/RP_7.csv
Group 8 records saved to RP/RP_8.csv
Group 9 records saved to RP/RP_9.csv
Group 10 records saved to RP/RP_10.csv
Group 11 records saved to RP/RP_11.csv
Group 12 records saved to RP/RP_12.csv
Group 13 records saved to RP/RP_13.csv
Group 14 records saved to RP/RP_14.csv
Group 15 records saved to RP/RP_15.csv
Group 16 records saved to RP/RP_16.csv
Group 17 records saved to RP/RP_17.csv
Group 18 records saved to RP/RP_18.csv
Group 19 records saved to RP/RP_19.csv
Group 20 records saved to RP/RP_20.csv
Group 21 records saved to RP/RP_21.csv
Group 22 records saved to RP/RP_22.csv
Group 23 records saved to RP/RP_23.csv
Group 24 records saved to RP/RP_24.csv
Group 25 records saved to RP/RP_25.csv
Group 

In [60]:
import re

rp_folder_path = 'RP'

rp_help_df = cudf.DataFrame(columns=['RP', 'LONGITUDE', 'LATITUDE', 'FLOOR'])

for file in os.listdir(rp_folder_path):
    if file.endswith('.csv'):
        file_number = int(re.search(r'RP_(\d+)\.csv', file).group(1))

        data = cudf.read_csv(os.path.join(rp_folder_path, file), nrows=1)

        longitude = data['LONGITUDE'].iloc[0]
        latitude = data['LATITUDE'].iloc[0]
        floor = data['FLOOR'].iloc[0]

        temp_df = cudf.DataFrame({'RP': [file_number], 'LONGITUDE': [longitude],
                                  'LATITUDE': [latitude], 'FLOOR': [floor]})

        rp_help_df = cudf.concat([rp_help_df, temp_df], ignore_index=True)

output_csv_path = '/content/RP_Help.csv'
rp_help_df.to_csv(output_csv_path)

print(output_csv_path)


/content/RP_Help.csv


In [61]:
noise_value = 0.05
mask_probability=0.05
mask_value=-104
augmentation = 5200

In [62]:
import numpy as np
from sklearn.utils import shuffle
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def add_noise(tensor):
    noise = torch.randn_like(tensor[:, :520], device=device) * noise_value
    tensor[:, :520] += noise
    return tensor

def mask_features(tensor, mask_probability=mask_probability, mask_value=mask_value):
    mask = (torch.rand(size=(tensor.size(0), 520), device=device, dtype=torch.float32) < mask_probability).float()
    tensor[:, :520] *= mask
    tensor[:, :520] += (1 - mask) * mask_value
    return tensor

input_folder = 'RP'
output_folder = 'RP_Augmented'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filepath in os.listdir(input_folder):
    if filepath.endswith('.csv'):
        df = pd.read_csv(os.path.join(input_folder, filepath))

        num_copies = augmentation // len(df)

        augmented_data = []

        for i in range(num_copies):
            df_values = df.values
            tensor_data = torch.tensor(df_values, device=device, dtype=torch.float32)

            augmented_tensor = add_noise(tensor_data.clone())
            augmented_tensor = mask_features(augmented_tensor.clone(), mask_probability=mask_probability)
            augmented_tensor[:, 520:] = tensor_data[:, 520:]

            augmented_data.append(augmented_tensor.cpu().numpy())

        augmented_data = np.concatenate(augmented_data, axis=0)
        augmented_data = shuffle(augmented_data)

        output_path = os.path.join(output_folder, f'augmented_{filepath}')
        pd.DataFrame(augmented_data, columns=df.columns).to_csv(output_path, index=False)

        print(f'Augmented {filepath} from {len(df)} to {len(augmented_data)} rows for {num_copies} copies')

print('Data augmentation complete for all files')


Augmented RP_56.csv from 42 to 5166 rows for 123 copies
Augmented RP_7.csv from 20 to 5200 rows for 260 copies
Augmented RP_53.csv from 20 to 5200 rows for 260 copies
Augmented RP_0.csv from 20 to 5200 rows for 260 copies
Augmented RP_44.csv from 20 to 5200 rows for 260 copies
Augmented RP_60.csv from 40 to 5200 rows for 130 copies
Augmented RP_17.csv from 20 to 5200 rows for 260 copies
Augmented RP_22.csv from 20 to 5200 rows for 260 copies
Augmented RP_35.csv from 38 to 5168 rows for 136 copies
Augmented RP_19.csv from 20 to 5200 rows for 260 copies
Augmented RP_1.csv from 30 to 5190 rows for 173 copies
Augmented RP_37.csv from 30 to 5190 rows for 173 copies
Augmented RP_57.csv from 47 to 5170 rows for 110 copies
Augmented RP_43.csv from 39 to 5187 rows for 133 copies
Augmented RP_63.csv from 40 to 5200 rows for 130 copies
Augmented RP_51.csv from 20 to 5200 rows for 260 copies
Augmented RP_5.csv from 30 to 5190 rows for 173 copies
Augmented RP_33.csv from 20 to 5200 rows for 260 cop

In [63]:
from cuml.preprocessing import MinMaxScaler

source_dir = '/content/RP_Augmented'
target_dir = '/content/RP_Normalized'
stats_dir = '/content/RP_Stats'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)
if not os.path.exists(stats_dir):
    os.makedirs(stats_dir)

def process_file(file_path, target_path, stats_path):
    df = cudf.read_csv(file_path)

    first_520_columns = df.iloc[:, :520]
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(first_520_columns)
    scaled_df = cudf.DataFrame(scaled_data)
    scaled_df.columns = first_520_columns.columns

    min_values = first_520_columns.min().to_pandas().values
    scale_values = scaler.scale_

    stats_df_corrected = cudf.DataFrame({'Column': first_520_columns.columns,
                                         'Min': min_values,
                                         'Scale': scale_values})

    remaining_columns = df.iloc[:, 520:]
    df_normalized = cudf.concat([scaled_df, remaining_columns], axis=1)

    chunksize = 10000
    df_normalized.to_csv(target_path, index=False, chunksize=chunksize)

    stats_df_corrected.to_csv(stats_path, index=False)

for filename in os.listdir(source_dir):
    if filename.endswith('.csv'):
        file_path = os.path.join(source_dir, filename)
        target_path = os.path.join(target_dir, f'normalized_{filename}')
        stats_path = os.path.join(stats_dir, f'stats_{filename}')
        process_file(file_path, target_path, stats_path)
        print("Processed file: ", filename)

Processed file:  augmented_RP_9.csv
Processed file:  augmented_RP_60.csv
Processed file:  augmented_RP_33.csv
Processed file:  augmented_RP_14.csv
Processed file:  augmented_RP_0.csv
Processed file:  augmented_RP_16.csv
Processed file:  augmented_RP_24.csv
Processed file:  augmented_RP_62.csv
Processed file:  augmented_RP_37.csv
Processed file:  augmented_RP_13.csv
Processed file:  augmented_RP_50.csv
Processed file:  augmented_RP_51.csv
Processed file:  augmented_RP_58.csv
Processed file:  augmented_RP_54.csv
Processed file:  augmented_RP_26.csv
Processed file:  augmented_RP_53.csv
Processed file:  augmented_RP_18.csv
Processed file:  augmented_RP_41.csv
Processed file:  augmented_RP_3.csv
Processed file:  augmented_RP_40.csv
Processed file:  augmented_RP_42.csv
Processed file:  augmented_RP_19.csv
Processed file:  augmented_RP_55.csv
Processed file:  augmented_RP_5.csv
Processed file:  augmented_RP_28.csv
Processed file:  augmented_RP_27.csv
Processed file:  augmented_RP_11.csv
Proce

In [64]:
import torch
import torch.nn as nn

class EnhancedStackedDenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(EnhancedStackedDenoisingAutoencoder, self).__init__()

        # Default
        input_dim = 520
        output_dim = 520
        layer_sizes = [256, 128, 64]
        noise_level = 0.01
        activation = 'ReLU'
        use_dropout = True
        dropout_prob = 0.4

        # Activation Function
        if activation == 'ReLU':
            self.activation = nn.ReLU()
        elif activation == 'LeakyReLU':
            self.activation = nn.LeakyReLU(0.2)

        # Encoder
        encoder_layers = []
        previous_size = input_dim
        for size in layer_sizes:
            encoder_layers.append(nn.Linear(previous_size, size))
            encoder_layers.append(nn.BatchNorm1d(size))
            encoder_layers.append(self.activation)
            if use_dropout:
                encoder_layers.append(nn.Dropout(dropout_prob))
            previous_size = size
        self.encoder = nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        for size in reversed(layer_sizes[:-1]):
            decoder_layers.append(nn.Linear(previous_size, size))
            decoder_layers.append(nn.BatchNorm1d(size))
            decoder_layers.append(self.activation)
            if use_dropout:
                decoder_layers.append(nn.Dropout(dropout_prob))
            previous_size = size
        decoder_layers.append(nn.Linear(previous_size, output_dim))
        decoder_layers.append(nn.Sigmoid())
        self.decoder = nn.Sequential(*decoder_layers)

        # سطح نویز
        self.noise_level = noise_level

    def forward(self, x):
        # اضافه کردن نویز
        noisy_x = x + torch.randn_like(x) * self.noise_level
        # رمزگذاری
        encoded = self.encoder(noisy_x)
        # رمزگشایی
        decoded = self.decoder(encoded)
        return decoded

In [65]:
def load_data(file_path):
    df = cudf.read_csv(file_path).to_pandas()
    X = df.iloc[:, :520].values

    X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

    train_tensor = torch.tensor(X_train, dtype=torch.float32)
    test_tensor = torch.tensor(X_test, dtype=torch.float32)

    train_dataset = TensorDataset(train_tensor, train_tensor)
    test_dataset = TensorDataset(test_tensor, test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    return train_loader, test_loader


In [66]:
import torch
import torch.nn as nn
import torch.optim as optim

def train_model(model, train_loader, test_loader, model_save_path, num_epochs=20):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for data, _ in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, data)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for data, _ in test_loader:
                data = data.to(device)
                outputs = model(data)
                loss = criterion(outputs, data)
                val_loss += loss.item()

        train_loss /= len(train_loader)
        val_loss /= len(test_loader)

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Validation Loss: {val_loss}')

    torch.save(model.state_dict(), model_save_path)

    plt.plot(train_losses, label='Training loss')
    plt.plot(val_losses, label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


In [67]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, TensorDataset,DataLoader
input_folder = '/content/RP_Normalized'
model_save_folder = 'trained_models'
os.makedirs(model_save_folder, exist_ok=True)

for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        model_save_path = os.path.join(model_save_folder, f'model_{file[:-4]}.pt')

        train_loader, test_loader = load_data(file_path)
        model = EnhancedStackedDenoisingAutoencoder()
        trained_model = train_model(model, train_loader, test_loader, model_save_path)

Output hidden; open in https://colab.research.google.com to view.

# Online

In [68]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import math
import os
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.metrics import mean_squared_error

In [69]:
training_data = pd.read_csv('/content/New_trainingData.csv')
rp_help_df =pd.read_csv("/content/RP_Help.csv")
test_data = pd.read_csv('/content/New_validationData.csv')
stats_folder = "/content/RP_Stats/"
model_folder = "/content/content/trained_models"

In [70]:
waps = 520
rps = rp_help_df.shape[0]
Weights =3
gamma =0
apply_normalization_check = False

In [71]:
def rbf_kernel(X, Y, gamma):
    pairwise_sq_dists = cdist(X, Y, 'sqeuclidean')
    return np.exp(-gamma * pairwise_sq_dists)

In [72]:
def estimate_probabilities(rbf_values):
    return rbf_values / rbf_values.sum(axis=1, keepdims=True)

In [73]:
def estimate_weighted_average(probabilities, coordinates):
    return np.dot(probabilities, coordinates)

In [74]:
def process_top_similarity(top, rp_help_data, gamma):
    top_10_df = pd.DataFrame(top, columns=['RP', 'Similarity'])
    merged_df = top_10_df.merge(rp_help_data, on='RP')
    coordinates = merged_df[['LONGITUDE', 'LATITUDE']].to_numpy()
    similarities = merged_df['Similarity'].to_numpy().reshape(-1, 1)
    rbf_values = rbf_kernel(similarities, similarities, gamma)
    probabilities = estimate_probabilities(rbf_values)
    weighted_avg_coordinates = estimate_weighted_average(probabilities, coordinates)
    return weighted_avg_coordinates

In [75]:
def load_model(model_path):
    model = EnhancedStackedDenoisingAutoencoder()
    model_state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model.load_state_dict(model_state_dict)
    model.eval()
    return model

In [76]:
def calculate_euclidean_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [77]:
def normalize_input_vector(input_vector, stats_df, apply_normalization_check):
    if isinstance(input_vector, pd.DataFrame):
        input_df = input_vector
    else:
        input_df = pd.DataFrame(input_vector, columns=stats_df['Column'].values)

    normalized_vector = input_df.copy()
    for column, min_val, scale in zip(stats_df['Column'].values, stats_df['Min'].values, stats_df['Scale'].values):
        normalized_vector[column] = (input_df[column] - min_val) * scale

    if apply_normalization_check:
        normalized_vector = normalized_vector.where((normalized_vector >= 0) & (normalized_vector <= 1), 0.5)

    return normalized_vector


In [78]:
def sort_by_distance(df, actual_long, actual_lat):
    df['Distance'] = df.apply(lambda row: round(calculate_euclidean_distance(actual_long, actual_lat,
                                                                       row['LONGITUDE'], row['LATITUDE']), 2), axis=1)
    sorted_df = df.sort_values(by='Distance')
    output = sorted_df[['RP', 'FLOOR', 'Distance']]
    return output

In [79]:
def process_data(rss_tensor, stats_file, model_file):
    stats_df = pd.read_csv(stats_file)
    normalized_vector = normalize_input_vector(rss_tensor, stats_df,apply_normalization_check)

    normalized_vector_np = normalized_vector.to_numpy()

    model = load_model(model_file)
    data_tensor = torch.tensor(normalized_vector_np, dtype=torch.float32)
    with torch.no_grad():
        reconstructed = model(data_tensor).cpu().numpy()

    mse = mean_squared_error(normalized_vector_np, reconstructed)
    return mse

In [80]:
import matplotlib.pyplot as plt
import numpy as np

stats_folder = "/content/RP_Stats/"
model_folder = "/content/trained_models"

distances_list_org = []

for i in range(len(test_data)):
    sample = test_data.iloc[[i]]
    rss_vector = sample.iloc[:, :waps].values
    actual_long = sample.iloc[:, waps].values[0]
    actual_lat = sample.iloc[:, waps+1].values[0]
    actual_floor = sample.iloc[:, waps+2].values[0]
    rss_tensor = torch.tensor(rss_vector, dtype=torch.float32)

    sorted_distances = sort_by_distance(rp_help_df, actual_long, actual_lat)
    sorted_distances.head()

    results = []

    for j in range(rps):
        stats_file = os.path.join(stats_folder, f"stats_augmented_RP_{j}.csv")
        model_file = os.path.join(model_folder, f"model_normalized_augmented_RP_{j}.pt")
        mse = process_data(rss_tensor, stats_file, model_file)
        results.append((j, mse))

    results.sort(key=lambda x: x[1])
    top = results[:Weights]

    weighted_avg_coordinates = process_top_similarity(top, rp_help_df, gamma)
    distance = calculate_euclidean_distance(actual_lat, actual_long, weighted_avg_coordinates[0][1], weighted_avg_coordinates[0][0])

    distances_list_org.append(distance)
    print(f"The difference for the test point {i} has been calculated.")
mean_distance = np.mean(distances_list_org)

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(distances_list_org)+1), distances_list_org, marker='o', linestyle='-', color='b', label='Distance')
plt.axhline(y=mean_distance, color='r', linestyle='--', label='Mean Distance')
#plt.title('Distance vs Iteration')
plt.xlabel('Index of test points')
plt.ylabel('Distance (meters)')
plt.legend()
plt.grid(True)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/trained_models/model_normalized_augmented_RP_0.pt'

In [ ]:
print(f"The average error for all reference points categorized by floor: {FLOOR} and building: {BUILDINGID} is equal to {mean_distance} meters.")
